In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

key = os.getenv("KOSIS_API_KEY")


In [ ]:
import json
import re


def parse_kosis_loose(text: str):
    """
    KOSIS에서 내려오는 [{OBJ_ID:"A", ...}] 형태(키 따옴표 없음)를
    JSON으로 보정해서 list[dict]로 변환
    """
    t = text.strip()

    # {KEY: 또는 ,KEY: 를 {"KEY": 로 변환
    t = re.sub(r"([{,]\s*)([A-Za-z0-9_]+)\s*:", r'\1"\2":', t)

    # 혹시 값이 '...'로 오는 경우까지 대비(현재는 "..."라 그대로 OK)
    t = t.replace("'", '"')

    return json.loads(t)


In [ ]:
import pandas as pd
import requests as req

import lib.util as u

base_url = "https://kosis.kr/openapi/statisticsData.do"

params = {
    "method": "getMeta",
    "type": "ITM",
    "apiKey": key,
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "format": "json",
}

res = req.get(base_url, params=params)

print(res.status_code)

res_text = res.text

print(res_text)

rows = u.parse_kosis_loose(res_text)
df = pd.DataFrame(rows)
# df.to_csv("../../data/item_code.csv")

In [ ]:
import requests as req

url = "https://kosis.kr/openapi/Param/statisticsParameterData.do"
params = {
    "method": "getList",
    "apiKey": key,
    "orgId": "101",
    "tblId": "DT_1YL20642",
    "itmId": "T001",
    "objL1": "ALL",
    "objL2": "ALL",  # 필수
    "prdSe": "Y",
    "newEstPrdCnt": "3",
    "format": "json",
    "jsonVD": "Y",  # 이거 안넣으면 json 형태로 안옴
}

res = req.get(url, params=params)
print(res.status_code)
print(res.text[:300])


In [ ]:
import pandas as pd

df = pd.DataFrame(res.json())
# df.head()

df.info()

df.describe()

In [ ]:
df

In [ ]:
# =========================
# 1. 데이터 타입 정리
# =========================
df["DT"] = pd.to_numeric(df["DT"], errors="coerce")
df["PRD_DE"] = df["PRD_DE"].astype(str)

# =========================
# 2. 청년순이동률만 선택
# =========================
df = df[df["C2"] == "11"]

# =========================
# 3. 수도권 + 광역시 제거
# =========================
exclude_codes = [
    "11",
    "23",
    "31",  # 수도권
    "21",
    "22",
    "24",
    "25",
    "26",  # 광역시
]

df = df[~df["C1"].astype(str).isin(exclude_codes)]

# =========================
# 4. 최신연도 TOP5
# =========================
latest_year = df["PRD_DE"].max()

top_5 = (
    df[df["PRD_DE"] == latest_year][["C1_NM", "DT"]]
    .rename(columns={"C1_NM": "지역", "DT": "청년순이동률"})
    .sort_values("청년순이동률", ascending=False)
    .head()
)

print("⭐ 최신연도:", latest_year)
print(top_5)

In [ ]:
import koreanize_matplotlib  # noqa: F401
import matplotlib.pyplot as plt

# =========================
# 6️⃣ 그래프
# =========================
x = range(len(top_5))
labels = top_5["지역"].tolist()
plt.figure()
plt.bar(x, top_5["청년순이동률"])
plt.xticks(x, labels, rotation=45)
# plt.bar(top_5["지역"], top_5["청년순이동률"])
plt.title(f"청년순이동률 TOP5 (수도권 + 광역시 제외, {latest_year})")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()